In [24]:
# Configuración de AMPL y licencia
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook

# Configurar licencia de AMPL y módulos necesarios
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"],
    license_uuid="bb2c71f6-2b4c-4570-8eaf-4db0b0d7349a"
)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\gabop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Licensed to Bundle #6733.7184 expiring 20241231: ICI5142-01 INVESTIGACION DE OPERACIONES AVANZADAS (Advanced Operations Research), Prof. Guillermo Cabrera-Guerrero, Pontificia Universidad Catolica de Valparaiso.


In [25]:
%%writefile sscflp.mod
# Definición de parámetros
param n; # Número de clientes
param m; # Número de centros de distribución
param c{i in 1..n, j in 1..m}; # Costo de transporte del cliente i al centro j
param f{j in 1..m}; # Costo fijo de apertura del centro j
param d{i in 1..n}; # Demanda del cliente i
param b{j in 1..m}; # Capacidad del centro j

# Definición de variables de decisión
var x{j in 1..m} binary; # Si el centro j está abierto
var y{i in 1..n, j in 1..m} binary; # Si el cliente i es atendido por el centro j

# Función objetivo: Minimizar los costos totales
minimize TotalCost: sum{i in 1..n, j in 1..m} c[i,j] * y[i,j] + sum{j in 1..m} f[j] * x[j];

# Restricción: Cada cliente debe ser atendido por exactamente un centro
subject to AssignCliente{i in 1..n}: sum{j in 1..m} y[i,j] = 1;

# Restricción: La capacidad del centro no debe ser excedida
subject to Capacity{j in 1..m}: sum{i in 1..n} d[i] * y[i,j] <= b[j] * x[j];

# Restricción: Un cliente solo puede ser asignado a un centro si el centro está abierto
subject to OpenIfAssigned{i in 1..n, j in 1..m}: y[i,j] <= x[j];

Overwriting sscflp.mod


In [26]:
%%writefile mscflp.mod 
# Definición de parámetros
param m; # Número de centros de distribución
param n; # Número de clientes
param f{i in 1..m}; # Costo de apertura del centro i
param c{i in 1..m, j in 1..n}; # Costo de transporte del centro i al cliente j
param S{i in 1..m}; # Capacidad del centro i
param D{j in 1..n}; # Demanda del cliente j

# Definición de variables de decisión
var x{i in 1..m, j in 1..n} >= 0; # Cantidad de demanda del cliente j atendida por el centro i
var y{i in 1..m} binary; # Si el centro i está abierto

# Función objetivo: Minimizar los costos totales
minimize TotalCost: sum{i in 1..m} f[i] * y[i] + sum{i in 1..m, j in 1..n} c[i,j] * x[i,j];

# Restricción: La capacidad del centro no debe ser excedida
subject to Capacity{i in 1..m}: sum{j in 1..n} x[i,j] <= S[i];

# Restricción: La demanda de cada cliente debe ser satisfecha
subject to Demand{j in 1..n}: sum{i in 1..m} x[i,j] >= D[j];

# Restricción: Un centro solo puede satisfacer demanda si está abierto
subject to OpenIfAssigned{i in 1..m, j in 1..n}: x[i,j] <= S[i] * y[i];


Overwriting mscflp.mod


In [27]:
# Imports generales
import os

# Ruta de las instancias
instance_dir = "Problem set OR Library"

# Selección de instancia (descomentar una línea para elegir)
#instance_file = f"{instance_dir}/cap41.txt"
# instance_file = f"{instance_dir}/capa.txt"
instance_file = f"{instance_dir}/500x2000_1.txt"
# instance_file = f"{instance_dir}/5000x5000_1.txt"

# Nombre de la instancia para usar en archivos de salida
instance_name = os.path.basename(instance_file).split('.')[0]


"""# Ruta de la carpeta de instancias
instance_folder = "./Problem set OR Library"
# Listar todas las instancias en el directorio
instances = [f for f in os.listdir(instance_folder) if f.endswith(".txt")]"""


# Selección de problema
problem = "SSCFLP"  # Cambiar a "MSCFLP" si se desea resolver el problema MSCFLP
#problem = "MSCFLP"


In [28]:
import pandas as pd

# Función de carga de datos
def load_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        
    # Procesar cada parte del archivo
    m, n = map(int, lines[0].split())  # Número de ubicaciones (m) y clientes (n)
    capacities, fixed_costs, demands, assign_costs = [], [], [], []
    
    # Leer capacidades y costos fijos
    idx = 2
    for _ in range(m):
        capacity, fixed_cost = map(float, lines[idx].split())
        capacities.append(capacity)
        fixed_costs.append(fixed_cost)
        idx += 1
    
    # Leer demandas
    for _ in range(n):
        demand = float(lines[idx].strip())
        demands.append(demand)
        idx += 1
    
    # Leer matriz de costos de asignación
    for _ in range(m):
        costs = list(map(float, lines[idx].split()))
        assign_costs.append(costs)
        idx += 1
    
    return m, n, capacities, fixed_costs, demands, assign_costs


In [29]:
# Ejemplo de configuración y resolución del problema con AMPL
def solve_instance(m, n, capacities, fixed_costs, demands, assign_costs, method="Benders"):
    # Crear AMPL model instance
    ampl = AMPL()
    model_path = "sscflp.mod" if problem == "SSCFLP" else "mscflp.mod"
    ampl.read(model_path)
    
    # Cargar datos de la instancia en el modelo AMPL
    ampl.param['m'] = m
    ampl.param['n'] = n
    ampl.param['c'] = assign_costs
    ampl.param['f'] = fixed_costs
    ampl.param['d'] = demands
    ampl.param['b'] = capacities

    if method == "Benders":
        # Descomposición de Benders con Gurobi como solver
        ampl.option["solver"] = "gurobi"
    else:
        # Aplicar búsqueda local
        pass

    # Resolver
    ampl.solve()
    return ampl.getObjective("TotalCost").value()


In [33]:
# Exportar resultados
def export_results(instance_file, solution, output_folder="resultadosInstancias"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    with open(f"{output_folder}/{instance_name}_solution.txt", 'w') as f:
        f.write(f"Instancia: {instance_name}\n")
        f.write(f"Solución óptima: {solution}\n")


In [32]:
# Ejecución de todas las instancias
for instance_file in instances:
    m, n, capacities, fixed_costs, demands, assign_costs = load_instance(os.path.join(instance_dir, instance_file))
    solution = solve_instance(m, n, capacities, fixed_costs, demands, assign_costs, method="Benders")
    export_results(instance_file.split(".")[0], solution)
